# Airdrop Simulator using NetworkX

### The implementation we're working towards is a monte-carlo style simulation to model our particular Airdrop

In [94]:
import networkx as nx
import random

class ELT2AirdropSimulation:    

    distributorProbibility = 0.5
    
    @staticmethod
    def create_initial_nodes(num_inital_nodes, of_which_are_distributors):
        """
        Creates the initial set of nodes in the graph.
        """
        G = nx.DiGraph()

        for i in range(num_inital_nodes):
            G.add_node(i, reputation=random.uniform(-1, 1), stake=random.uniform(0, 1), opt_in=random.choice([True, False]), stage=0, distributor=random.choice([True, False]))
            if i == 0:
                this_node = G.nodes[i]
                this_node['stake'] = 1000
                this_node['opt_in'] = True
                this_node['reputation'] = 1
                this_node['distributor'] = True
                this_node['AIRDROP CONTRACT'] = True
            else:
                G.add_edge(0, i, edge_type='contract_airdrop')
        return G


    @staticmethod
    def iterate_data_set(G, num_steps, randomnessFactor):
        """
        Iterates through the dataset for a specified number of steps.
        """
        for i in range(num_steps):
            node_iter = 0
            for node in list(G.nodes()):
                if node_iter == 0:
                    node_iter = node_iter +1;
                    None
                else:
                    ELT2AirdropSimulation.updateNode(G, node, randomnessFactor)
                    ELT2AirdropSimulation.updateDistNode(G, node, randomnessFactor)
                    ELT2AirdropSimulation.increment_distributor_node_edge(G, node, randomnessFactor)
                    node_iter = node_iter +1;
        return G
    

    @staticmethod
    def updateNode(G, node, randomnessFactor):
        ELT2AirdropSimulation.increment_node_reputation(G, node, randomnessFactor)
        ELT2AirdropSimulation.increment_node_stake(G, node, randomnessFactor)
        ELT2AirdropSimulation.increment_node_opt_in(G, node, randomnessFactor)
        ELT2AirdropSimulation.increment_node_stage(G, node)
        ELT2AirdropSimulation.set_distributor_node_status(G, node)
    @staticmethod
    def updateDistNode(G, node, randomnessFactor):
         ELT2AirdropSimulation.increment_distributor_node_edge(G, node, randomnessFactor)
            
    @staticmethod
    def set_distributor_node_status(G, node):
        """
        Iterates through each node in the graph and checks if the node meets the eligibility criteria to be a distributor.
        """
        if ELT2AirdropSimulation.check_distributor_probability(G, node):
            if ELT2AirdropSimulation.check_distributor_eligibility(G, node):
                # Adding a promote-distributor edge to the graph representing contract event where a node is promoted to a distributor
                G.add_edge(0, node, edge_type='promote-distributor')
                # Set the Distributor attribute on the node to True
                G.nodes()[node]['distributor'] = True
            else:
                G.nodes()[node]['distributor'] = False
    @staticmethod
    def check_distributor_eligibility(G, node):
        """
        Checks if a node meets the eligibility criteria to be a distributor.
        """
        return G.nodes[node]['reputation'] > 0.8 and G.nodes[node]['stake'] > 0.8 and G.nodes[node]['opt_in'] == True #and G.has_edge(0, node)
    
    @staticmethod
    def check_distributor_probability(G, node):
        """
        Checks if a node meets the probability criteria to be a distributor.
        """
        return random.uniform(0, 1) < ELT2AirdropSimulation.distributorProbibility
    
    @staticmethod
    def add_airdrop_edge(G, node):
        """
        Adds a new node to the graph, connecting it to the input node via an "airdrop" edge.
        """
        new_node = len(G.nodes())
        G.add_node(new_node, reputation=random.uniform(0, 1), stake=random.uniform(0, 1), opt_in=random.choice([True, False]), stage=0)

        ELT2AirdropSimulation.set_distributor_node_status(G, new_node)
        
        try:
            if G.nodes()[new_node]['distributor'] == True : 
                isDist = True
            else:
                isDist = False
        except:
            G.nodes()[new_node]['distributor'] = False
        
        ELT2AirdropSimulation.updateNode(G, new_node, randomnessFactor)
        if G.has_edge(0, new_node) and isDist == False :
            G.add_edge(0, new_node, edge_type='initial_airdrop')            
        else :
#             G.add_edge(node, new_node, edge_type='distributor_airdrop')
              None
    @staticmethod
    def increment_distributor_node_edge(G, _node,  _randomness):
        """
        Increments all distributor nodes, spawning edges to new nodes from the
        particular distributor according to the random value multiplied by the
        randomness attribute.
        """
        if G.nodes()[_node].get('distributor') and random.uniform(0, 1) < _randomness:
            new_node = len(G.nodes())
            G.add_node(new_node, reputation=random.uniform(-1, 1), stake=random.uniform(0, 1), opt_in=random.choice([True, False]), stage=0)
            G.add_edge(_node, new_node, edge_type='distributor_airdrop')
            ELT2AirdropSimulation.set_distributor_node_status(G, new_node)
    def increment_contract_airdrop_edge(G, _node,  _randomness):
        """
        Increments all distributor nodes, spawning edges to new nodes from the
        particular distributor according to the random value multiplied by the
        randomness attribute.
        """
        if random.uniform(0, 1) < _randomness:
            new_node = len(G.nodes())
            G.add_node(new_node, reputation=random.uniform(-1, 1), stake=random.uniform(0, 1), opt_in=random.choice([True, False]), stage=0)
            G.add_edge(0, new_node, edge_type='contract_airdrop')
            ELT2AirdropSimulation.set_distributor_node_status(G, new_node)
            
                
    @staticmethod
    def increment_node_reputation(G, node, randomness):
        """
        Increments a node's reputation attribute based on a random value multiplied by the randomness attribute.
        """
        G.nodes[node]['reputation'] += (random.uniform(-1, 1) * randomness) * (1 + G.nodes[node].get('distributor', False))

    @staticmethod
    def increment_node_stake(G, node, randomness):
        """
        Increments a node's stake attribute based on a random value multiplied by the randomness attribute.
        """
        G.nodes[node]['stake'] += (random.uniform(0, 1) * randomness) * (1 + G.nodes[node].get('distributor', False))

    @staticmethod
    def increment_node_opt_in(G, node, randomness):
        """
        Increments a node's opt_in attribute based on a random value multiplied by the randomness attribute.
        """
        if G.nodes[node]['opt_in'] == False:
            G.nodes[node]['opt_in'] = bool(random.uniform(0, 1) + randomness > 0.5)
        else:
            None


    @staticmethod
    def increment_node_stage(G, node):
        """
        Increments a node's stage attribute by 1
        """
        G.nodes[node]['stage'] += 1

In [134]:
# Required setup and runner methods...

from ipysigma import Sigma, SigmaGrid


#Phi = Golden ratio (Maybe a dumb idea)
phi = 1.6180339887498948482045868343656381177203091798057628621354486227
randomnessFactor = phi


def printDatasetKeyMetrics(G):
    """
    Prints out key metrics from the graph object
    """
    print("Number of Nodes: ", G.number_of_nodes())
    print("Number of Edges: ", G.number_of_edges())
    print("Number of Distributors: ", len([n for n in G.nodes() if G.nodes[n].get("distributor")]))
    print("Average node in-degree: ", sum(d for n, d in G.in_degree()) / len(G))
    print("Average node out-degree: ", sum(d for n, d in G.out_degree()) / len(G))
    
def doDatasetIteration(_G, _iter):
    G = incrementGraph(_G, _iter, randomnessFactor)
    printDatasetKeyMetrics(_G)
    
def incrementGraph(_G, iterations, randomnessFactor):
    return ELT2AirdropSimulation.iterate_data_set(_G, iterations, randomnessFactor)
    print("\nIterations:", iterations, "\nRandomness Factor:", randomnessFactor, "\n")

def renderSigmaGraph(_G):
    return Sigma(
        _G,
        node_size_range=(2,8),
        node_size='stake',
        node_color='reputation',
        node_color_gradient='Viridis',
        node_label="distributor",
        default_edge_type="curve",
        node_border_color_from="node",
        edge_color='edge_type',
        edge_color_palette='Set1',
        #{
         #   'contract_airdrop': 'green',
          #  'distributor_airdrop': 'red',
           # 'initial_airdrop':'blue',
            #'promote-distributor':'purple'
        #},
        edge_label="edge_type"
    )
def renderSigmaGrid(_G):
    return SigmaGrid(
        _G,
        node_color_gradient='Viridis',
        node_color='reputation', edge_label='edge_type').add(node_size='stake').add(node_size='reputation')

## Setup our simulation
### This includes the number of inital nodes, and the number of iterations to do this cycle
---

In [135]:
#Define a graph from the inital nodes set here



num_nodes = 25
iterations = 0

G = ELT2AirdropSimulation.create_initial_nodes(num_nodes, 0.3)
printDatasetKeyMetrics(G)

Number of Nodes:  25
Number of Edges:  24
Number of Distributors:  14
Average node in-degree:  0.96
Average node out-degree:  0.96


### The graph at initial nodes, with 0 iterations of the dataset model.

In [139]:
renderSigmaGraph(G)

Sigma(nx.DiGraph with 177 nodes and 198 edges)

## After 5 Iterations
*Re run the above cell after the next*


In [138]:
iterations = 5
doDatasetIteration(G, iterations)

Number of Nodes:  177
Number of Edges:  198
Number of Distributors:  27
Average node in-degree:  1.11864406779661
Average node out-degree:  1.11864406779661


In [67]:
renderSigmaGrid(G)